In [1]:
# Hybrid model van Roos namaken
# Combinatie van OpenFace (FaceNet) en CFPS


# Face representatie uit gezichten halen


# Test set uit Labelled Faces in the Wild voor pipeline



In [1]:
import time

start = time.time()

import argparse
import cv2
import itertools
import os

import numpy as np
np.set_printoptions(precision=2)

import openface
import dlib

In [2]:
# From https://github.com/cmusatyalab/openface/blob/master/demos/compare.py

# fileDir = r"C:\Users\Fien\Documents\Universiteit\M2\AI-Thesis\openface" # os.path.dirname(os.path.realpath(__file__))
# modelDir = os.path.join(fileDir, '\models')
# dlibModelDir = os.path.join(modelDir, '\dlib')
# openfaceModelDir = os.path.join(modelDir, '\openface')
# modelDir = r"C:\Users\Fien\Documents\Universiteit\M2\AI-Thesis\openface\models"
# dlibModelDir = r"C:\Users\Fien\Documents\Universiteit\M2\AI-Thesis\openface\models\dlib"
# openfaceModelDir = r"C:\Users\Fien\Documents\Universiteit\M2\AI-Thesis\openface\models\openface"

imgs = [] # input images
dlibFacePredictor = "/home/fien/openface/models/dlib/shape_predictor_68_face_landmarks.dat"
networkModel = "/home/fien/openface/models/openface/nn4.small2.v1.t7"
# networkModel = os.path.join(openfaceModelDir, "/openface.nn4.small2.v1.t7")
# networkModel = r"C:\Users\Fien\Documents\Universiteit\M2\AI-Thesis\openface\models\openface\openface.nn4.small2.v1.t7"

imgDim = 96
verbose = True

if verbose:
    print("Argument parsing and loading libraries took {} seconds.".format(
        time.time() - start))

start = time.time()
align = openface.AlignDlib(dlibFacePredictor)
net = openface.TorchNeuralNet(networkModel, imgDim)
   
    
if verbose:
    print("Loading the dlib and OpenFace models took {} seconds.".format(
        time.time() - start))

Argument parsing and loading libraries took 35.308371067 seconds.
True


In [3]:
def getRep(imgPath):
    if verbose:
        print("Processing {}.".format(imgPath))
    bgrImg = cv2.imread(imgPath)
    if bgrImg is None:
        raise Exception("Unable to load image: {}".format(imgPath))
    rgbImg = cv2.cvtColor(bgrImg, cv2.COLOR_BGR2RGB)

    if verbose:
        print("  + Original size: {}".format(rgbImg.shape))

    start = time.time()
    bb = align.getLargestFaceBoundingBox(rgbImg)
    if bb is None:
        raise Exception("Unable to find a face: {}".format(imgPath))
    if verbose:
        print("  + Face detection took {} seconds.".format(time.time() - start))

    start = time.time()
    alignedFace = align.align(args.imgDim, rgbImg, bb,
                              landmarkIndices=openface.AlignDlib.OUTER_EYES_AND_NOSE)
    if alignedFace is None:
        raise Exception("Unable to align image: {}".format(imgPath))
    if verbose:
        print("  + Face alignment took {} seconds.".format(time.time() - start))

    start = time.time()
    rep = net.forward(alignedFace)
    if verbose:
        print("  + OpenFace forward pass took {} seconds.".format(time.time() - start))
        print("Representation:")
        print(rep)
        print("-----\n")
    return rep

In [ ]:
img1 = 
img2 = 

for (img1, img2) in itertools.combinations(args.imgs, 2):
    d = getRep(img1) - getRep(img2)
    print("Comparing {} with {}.".format(img1, img2))
    print(
        "  + Squared l2 distance between representations: {:0.3f}".format(np.dot(d, d)))